In [ ]:
import ROOT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import math
import uproot
from array import array

print("Got imports")

In [ ]:
#data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndhist_data_nocalib_6k.root"

#data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndwidths_data_xf_6k.root"
#mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndwidths_mc_xf_6k.root" 

data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/ndhists_data_space_nocalib_4k.root"
mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/ndhists_mc_space_nocalib_5k.root"

print("Got File to analyze")

DATA_TAG = "v10_06_00_02"

MC_SCALE = 2.75

# Helper Functions

In [ ]:
# I want to convert a ROOT TH2D to a plt.hist2d for better plotting
def root_th2d_to_plt_hist2d(th2d):
    # Get the bin edges and contents from the ROOT TH2D
    x_edges = [th2d.GetXaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsX() + 2)]
    y_edges = [th2d.GetYaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsY() + 2)]
    z_values = np.zeros((th2d.GetNbinsX(), th2d.GetNbinsY()))

    for i in range(1, th2d.GetNbinsX() + 1):
        for j in range(1, th2d.GetNbinsY() + 1):
            z_values[i-1, j-1] = th2d.GetBinContent(i, j)

    return np.array(x_edges), np.array(y_edges), np.array(z_values)

def get_1d_edges(h):
    x_edges = [h.GetXaxis().GetBinLowEdge(i) for i in range(1, h.GetNbinsX() + 2)] 
    return x_edges

# Example usage:
# th2d = rfile_data.Get("hwidth0")
# x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
# plt.hist2d(x_edges[:-1], y_edges[:-1], z_values.T, bins=[x_edges, y_edges], cmap='viridis')
# plt.colorbar(label='Counts')
# plt.xlabel('X-axis label')                
#plt.ylabel('Y-axis label')
# plt.title('2D Histogram from ROOT TH2D')
# plt.show()    

# I want to convert a ROOT TH1D with errors to a plt.errorbar for better plotting
def root_th1d_to_plt_errorbar(th1d):
    # Get the bin centers, contents, and errors from the ROOT TH1D
    x_values = [th1d.GetBinCenter(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_values = [th1d.GetBinContent(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_errors = [th1d.GetBinError(i) for i in range(1, th1d.GetNbinsX() + 1)]

    return np.array(x_values), np.array(y_values), np.array(y_errors) 

# Example usage:
# th1d = rfile_data.Get("hwidth0").ProjectionX()
# x_values, y_values, y_errors = root_th1d_to_plt_errorbar(th1d)
# plt.errorbar(x_values, y_values, yerr=y_errors, fmt='o', ecolor='red', capsize=5)
# plt.xlabel('X-axis label')                
# plt.ylabel('Y-axis label')
# plt.title('1D Histogram with Error Bars from ROOT TH1D')
# plt.show()


# Function to create a ROOT TH1D from means and errors
def make_mean_hist(h2d, means, errors):
    h_means = h2d.ProjectionX()
    h_means.Reset()
    for i in range(1, h_means.GetNbinsX() + 1):
        h_means.SetBinContent(i, means[i-1])
        h_means.SetBinError(i, errors[i-1])
    
    return h_means

In [ ]:
def get_track_hist_1d(f, dim, tpc, plane, var):
    idx = 3*tpc + plane
    rfile = ROOT.TFile.Open(f, "READ")
    h2d = rfile.Get("hntrk_"+str(idx)+"_"+var)
    h1d = h2d.ProjectionX("h1d_"+str(idx)+"_"+var+"_"+f)
    h1d.SetDirectory(0)
    rfile.Close()
    return h1d


# 1D Plotting

In [ ]:
# Test Functionality

h = get_track_hist_1d(data_file, 0, 0, 0, "x")

c = ROOT.TCanvas("c", "c", 700, 500)
h.Draw("HISTE")
c.Draw()

In [ ]:
# Test Functionality

h = get_track_hist_1d(data_file, 2, 0, 2, "z")

c = ROOT.TCanvas("c", "c", 700, 500)
h.Draw("HISTE")
c.Draw()

In [ ]:

def plot_tracks_1d(f, DIM, VAR, xlim0=None, xlim1=None, ylim=None, DATA_TAG=None, xlabel=None):

    kNtpcs = 2
    kNplanes = 3

    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    gs = gridspec.GridSpec(1, 2 + 1, width_ratios=[1, 1, 0.05], wspace=0.1)

    largest_max_z = 0

    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane

            h = get_track_hist_1d(f, DIM, tpc, plane, VAR)

            x_edges = get_1d_edges(h)

            ax = axs[tpc, plane]
            
            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)

            if ylim is not None:
                ax.set_ylim(ylim)

            ax.set_title(f'TPC {tpc}, Plane {plane}')


            x, y, yerr = root_th1d_to_plt_errorbar(h)
           
            ax.errorbar(x, y/(10**6), xerr=np.ones_like(yerr)*((x_edges[1]-x_edges[0])/2), 
                        yerr=yerr/(10**6), fmt='o', ecolor='black', capsize=0, label='Data', markersize=0)
           
            ax.grid()
            #ax.legend()
            
            h.Delete()
           

   
            
    if xlabel is not None:
        fig.supxlabel(xlabel, fontsize=20)

    fig.supylabel("Number of Tracks "+r"(x$10^{-6}$)", fontsize=20)
    t = "SBND Off-Beam Calibration: "

    if DATA_TAG is not None:
        t += DATA_TAG
    fig.suptitle(t, fontsize=20)
    #cbar_ax = fig.add_subplot(gs[0, 2])
    #fig.colorbar(pcm, cax=cbar_ax)
    #fig.colorbar(pcm, ax=axs, fraction=.05, label='')
    plt.tight_layout()
    plt.show()

print("Defining plot function")

In [ ]:
plot_tracks_1d(data_file, 0, "x", xlim0=[-220, 20], xlim1=[-20, 220], ylim=None, DATA_TAG=DATA_TAG, xlabel="Reconstructed Hit X [cm]")

In [ ]:
plot_tracks_1d(data_file, 1, "y", xlim0=[-220, 220], xlim1=[-220, 220], ylim=None, DATA_TAG=DATA_TAG, xlabel="Reconstructed Hit Y [cm]")

In [ ]:
plot_tracks_1d(data_file, 2, "z", xlim0=[-20, 520], xlim1=[-20, 520], ylim=None, DATA_TAG=DATA_TAG, xlabel="Reconstructed Hit Z [cm]")

In [ ]:

def plot_tracks_1d_compare(DIM, VAR, xlim0=None, xlim1=None, ylim=None, DATA_TAG=None, xlabel=None, MC_SCALE=3):

    kNtpcs = 2
    kNplanes = 3

    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    gs = gridspec.GridSpec(1, 2 + 1, width_ratios=[1, 1, 0.05], wspace=0.1)

    largest_max_z = 0

    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane

            h = get_track_hist_1d(data_file, DIM, tpc, plane, VAR)
            h2 = get_track_hist_1d(mc_file, DIM, tpc, plane, VAR)

            x_edges = get_1d_edges(h)

            ax = axs[tpc, plane]
            
            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)

            if ylim is not None:
                ax.set_ylim(ylim)

            ax.set_title(f'TPC {tpc}, Plane {plane}')


            x, y, yerr = root_th1d_to_plt_errorbar(h)
            x2, y2, yerr2 = root_th1d_to_plt_errorbar(h2)
           
            ax.errorbar(x, y/(10**6), xerr=np.ones_like(yerr)*((x_edges[1]-x_edges[0])/2), 
                        yerr=yerr/(10**6), fmt='o', ecolor='black', capsize=0, label='Data', markersize=0, linewidth=2)
            ax.errorbar(x2, MC_SCALE*y2/(10**6), xerr=np.ones_like(yerr2)*((x_edges[1]-x_edges[0])/2), 
                        yerr=MC_SCALE*yerr2/(10**6), fmt='o', ecolor='red', capsize=0, label='MC (x'+str(MC_SCALE)+')', markersize=0, linewidth=2)
           
            ax.grid()
            if tpc == 0 and plane == 0:
                ax.legend(fontsize=12)
            
            h.Delete()
            h2.Delete()
           

   
            
    if xlabel is not None:
        fig.supxlabel(xlabel, fontsize=20)

    fig.supylabel("Number of Tracks "+r"(x$10^{-6}$)", fontsize=20)
    t = "SBND Off-Beam Calibration: "

    if DATA_TAG is not None:
        t += DATA_TAG
    fig.suptitle(t, fontsize=20)
    #cbar_ax = fig.add_subplot(gs[0, 2])
    #fig.colorbar(pcm, cax=cbar_ax)
    #fig.colorbar(pcm, ax=axs, fraction=.05, label='')
    plt.tight_layout()
    plt.show()

print("Defining plot function")

In [ ]:
plot_tracks_1d_compare(0, "x", xlim0=[-220, 20], xlim1=[-20, 220], ylim=None, DATA_TAG=DATA_TAG, xlabel="Reconstructed Hit X [cm]", MC_SCALE=MC_SCALE)

In [ ]:
plot_tracks_1d_compare(1, "y", xlim0=[-220, 220], xlim1=[-220, 220], ylim=None, 
                       DATA_TAG=DATA_TAG, xlabel="Reconstructed Hit Y [cm]", MC_SCALE=MC_SCALE)

In [ ]:
plot_tracks_1d_compare(2, "z", xlim0=[-20, 520], xlim1=[-20, 520], ylim=None, 
                       DATA_TAG=DATA_TAG, xlabel="Reconstructed Hit Z [cm]", MC_SCALE=MC_SCALE)

In [ ]:
plot_tracks_1d_compare(3, "txz", xlim0=[-90, 90], xlim1=[-90, 90], ylim=None, 
                       DATA_TAG=DATA_TAG, xlabel="Reconstructed "+r"$\theta_{XZ}$ [degrees]", MC_SCALE=MC_SCALE)

In [ ]:
plot_tracks_1d_compare(4, "tyz", xlim0=[-90, 90], xlim1=[-90, 90], ylim=None, 
                       DATA_TAG=DATA_TAG, xlabel="Reconstructed "+r"$\theta_{YZ}$ [degrees]", MC_SCALE=MC_SCALE)